# Room Utilization Analysis for Reed Mews 

Author - Vamsi Mudila (vamsi.mudila@gmail.com)

## Objective:
This project aims to:
1. Analyze room occupancy data for the ground and first floors of Reed Mews.
2. Calculate key metrics:
   - Room utilization levels (average, peak, underutilized, and overutilized).
   - Trends and patterns (daily and weekly usage).
   - Anomalies in room usage.
3. Provide evidence-based recommendations for space optimization.
4. Propose a scalable solution for ongoing monitoring and reporting.
5. Build an interactive dashboard to present insights interactively.

### Deliverables:
1. Data analysis and insights.
2. Interactive dashboard for live visualization.
3. Evidence-based recommendations.
4. Scalable data model for future monitoring.

## Step 1: Import Libraries and Load Data

We import essential Python libraries for data analysis, visualization, and interactivity. The occupancy datasets for the ground and first floors are loaded for analysis.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from tabulate import tabulate
from scipy.stats import zscore
import threading
import webbrowser

# Load datasets
ground_floor = pd.read_excel("Wellbeing Occupancy_DataSet.xlsx", sheet_name="Ground Floor", skiprows=1)
first_floor = pd.read_excel("Wellbeing Occupancy_DataSet.xlsx", sheet_name="First Floor", skiprows=1)

# Display random samples
print("Random 5 Rows from Ground Floor Data:")
print(tabulate(ground_floor.sample(5), headers="keys", tablefmt="fancy_grid"))

print("\nRandom 5 Rows from First Floor Data:")
print(tabulate(first_floor.sample(5), headers="keys", tablefmt="fancy_grid"))

## Step 2: Data Cleaning

To ensure accurate analysis:
1. Remove duplicate entries.
2. Check for missing values and handle them if necessary.
3. Format timestamps for time-based analysis.
4. Set timestamps as the index for easy grouping and trends analysis.

In [ ]:
# Remove duplicates
ground_floor.drop_duplicates(inplace=True)
first_floor.drop_duplicates(inplace=True)

# Check for missing values
ground_missing = ground_floor.isnull().sum().reset_index()
first_missing = first_floor.isnull().sum().reset_index()

# Format missing value data
ground_missing.columns = ["Column", "Missing Values"]
first_missing.columns = ["Column", "Missing Values"]

print("\nMissing Values in Ground Floor Data:")
print(tabulate(ground_missing, headers="keys", tablefmt="fancy_grid"))

print("\nMissing Values in First Floor Data:")
print(tabulate(first_missing, headers="keys", tablefmt="fancy_grid"))

# Format timestamps
ground_floor['Timestamp'] = pd.to_datetime(ground_floor['Timestamp'], dayfirst=True)
first_floor['Timestamp'] = pd.to_datetime(first_floor['Timestamp'], dayfirst=True)

# Set Timestamp as the index
ground_floor.set_index('Timestamp', inplace=True)
first_floor.set_index('Timestamp', inplace=True)

## Step 3: Key Metrics Calculation

We calculate:
1. Average utilization for each room to understand overall usage.
2. Peak utilization levels to identify rooms with high demand.
3. Underutilized (<30%) and overutilized (>90%) rooms for optimization insights.

In [ ]:
# Calculate average and peak utilization
ground_utilization = ground_floor.mean()
ground_peak = ground_floor.max()

first_utilization = first_floor.mean()
first_peak = first_floor.max()

# Define thresholds for underutilization and overutilization
UNDERUTILIZED_THRESHOLD = 0.3
OVERUTILIZED_THRESHOLD = 0.9

# Identify underutilized and overutilized rooms
ground_underutilized = ground_utilization[ground_utilization < UNDERUTILIZED_THRESHOLD]
ground_overutilized = ground_utilization[ground_utilization > OVERUTILIZED_THRESHOLD]

first_underutilized = first_utilization[first_utilization < UNDERUTILIZED_THRESHOLD]
first_overutilized = first_utilization[first_utilization > OVERUTILIZED_THRESHOLD]

# Display key metrics
print("\nGround Floor Utilization Metrics:")
print(tabulate(pd.DataFrame({'Room': ground_utilization.index, 
                             'Avg Utilization': ground_utilization.values, 
                             'Peak Utilization': ground_peak.values}), headers="keys", tablefmt="fancy_grid"))

print("\nFirst Floor Utilization Metrics:")
print(tabulate(pd.DataFrame({'Room': first_utilization.index, 
                             'Avg Utilization': first_utilization.values, 
                             'Peak Utilization': first_peak.values}), headers="keys", tablefmt="fancy_grid"))

## Step 4: Trends and Patterns Analysis

This step analyzes:
1. **Hourly Trends**: Understand utilization trends across different hours of the day.
2. **Weekly Trends**: Examine usage patterns by weekdays.
3. **Anomalies**: Use statistical methods to detect irregular patterns in room utilization.

In [19]:
import plotly.express as px

# Add time-based features for analysis
for df in [ground_floor, first_floor]:
    df['Hour'] = df.index.hour
    df['Weekday'] = df.index.day_name()

# Select only room columns for aggregation
room_columns_ground = [col for col in ground_floor.columns if col not in ['Hour', 'Weekday']]
room_columns_first = [col for col in first_floor.columns if col not in ['Hour', 'Weekday']]

# Aggregate by hour and weekday
ground_hourly = ground_floor[room_columns_ground].groupby(ground_floor['Hour']).mean()
first_hourly = first_floor[room_columns_first].groupby(first_floor['Hour']).mean()

ground_weekly = ground_floor[room_columns_ground].groupby(ground_floor['Weekday']).mean()
first_weekly = first_floor[room_columns_first].groupby(first_floor['Weekday']).mean()

# Sort weekly data for proper ordering (Monday to Sunday)
week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ground_weekly = ground_weekly.reindex(week_order)
first_weekly = first_weekly.reindex(week_order)

# Heatmap for time-based room utilization
ground_heatmap = ground_floor[room_columns_ground].groupby([ground_floor['Hour'], ground_floor['Weekday']]).mean()
first_heatmap = first_floor[room_columns_first].groupby([first_floor['Hour'], first_floor['Weekday']]).mean()

# Reset index for plotting
ground_heatmap = ground_heatmap.reset_index()
first_heatmap = first_heatmap.reset_index()

# Add z-score anomalies
ground_anomalies = (ground_floor[room_columns_ground] - ground_floor[room_columns_ground].mean()) / ground_floor[room_columns_ground].std()
first_anomalies = (first_floor[room_columns_first] - first_floor[room_columns_first].mean()) / first_floor[room_columns_first].std()


## Step 5: Reporting and Visualization
Create an interactive dashboard to present findings, allowing users to explore trends and insights in room utilization. The dashboard includes:
1. Room utilization levels (average, peak, underutilized, and overutilized).
2. Trends and patterns (hourly and weekly usage).
3. Filters to explore individual rooms or compare all rooms.

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Initialize the Dash app
app = Dash(__name__)

# Enhanced layout for the dashboard
app.layout = html.Div([
    # Header Section
    html.Div([
        html.H1("Room Utilization Dashboard", style={
            "text-align": "center", "font-size": "36px", "color": "#1F4E78",
            "margin-bottom": "10px"}),
        html.P("An interactive tool to analyze room usage patterns and optimize space utilization at Reed Mews.",
               style={"text-align": "center", "font-size": "18px", "color": "#444"})
    ], style={"padding": "10px", "background-color": "#F4F6F8", "border-bottom": "2px solid #1F4E78"}),

    # Filters for Floor and Room Selection
    html.Div([
        html.Div([
            html.Label("Select Floor:", style={"font-size": "16px", "font-weight": "bold"}),
            dcc.RadioItems(
                id="floor-selector",
                options=[
                    {"label": "Ground Floor", "value": "ground"},
                    {"label": "First Floor", "value": "first"}
                ],
                value="ground",
                inline=True,
                style={"margin-bottom": "15px"}
            )
        ], style={"width": "48%", "display": "inline-block", "vertical-align": "top"}),

        html.Div([
            html.Label("Select Room(s):", style={"font-size": "16px", "font-weight": "bold"}),
            dcc.Dropdown(
                id="room-selector",
                placeholder="Select a Room",
                multi=True,
                style={"width": "100%"}
            )
        ], style={"width": "48%", "display": "inline-block", "vertical-align": "top"})
    ], style={"padding": "10px", "background-color": "#E8F5E9", "border-radius": "5px", "margin": "20px 0"}),

    # Overview Section
    html.Div(id="summary-overview", style={
        "margin-bottom": "20px", "font-size": "16px", "font-weight": "bold",
        "padding": "15px", "background-color": "#FFF3E0", "border-radius": "5px",
        "border-left": "5px solid #FFB74D"
    }),

    # Visualizations Section
    dcc.Tabs([
        dcc.Tab(label="Hourly Trends", children=[
            html.Div([
                html.P("This graph shows the hourly utilization trends for the selected room(s) or all rooms. It helps identify times of high and low usage throughout the day, providing insights for better scheduling.",
                       style={"font-size": "14px", "margin-bottom": "10px"}),
                dcc.Graph(id="hourly-trends")
            ], style={"padding": "10px"})
        ]),
        dcc.Tab(label="Weekly Trends", children=[
            html.Div([
                html.P("This bar chart highlights room utilization trends for each day of the week. It aids in identifying high-demand days and supports better planning for resources and availability.",
                       style={"font-size": "14px", "margin-bottom": "10px"}),
                dcc.Graph(id="weekly-trends")
            ], style={"padding": "10px"})
        ]),
        dcc.Tab(label="Heatmap", children=[
            html.Div([
                html.P("The heatmap visualizes room utilization across hours of the day and days of the week. Darker colors represent higher usage, allowing you to spot busy periods at a glance.",
                       style={"font-size": "14px", "margin-bottom": "10px"}),
                dcc.Graph(id="heatmap")
            ], style={"padding": "10px"})
        ]),
        dcc.Tab(label="Anomalies", children=[
            html.Div([
                html.P("This scatter plot shows anomalies in room utilization. Points far from the baseline indicate unusual usage patterns that may require further investigation.",
                       style={"font-size": "14px", "margin-bottom": "10px"}),
                dcc.Graph(id="anomalies")
            ], style={"padding": "10px"})
        ])
    ], style={"margin-top": "20px", "background-color": "#F9F9F9", "border-radius": "5px"}),

    # Footer Section
html.Div([
    html.P([
        "Vamsi Mudila | ",
        html.A("vamsi.mudila@gmail.com", href="mailto:vamsi.mudila@gmail.com", style={"color": "#1F4E78", "text-decoration": "none"})
    ], style={
        "text-align": "center", "font-size": "14px", "color": "#555",
        "padding": "10px", "margin-top": "20px", "background-color": "#F4F6F8",
        "border-top": "2px solid #1F4E78"
        })
    ])
])


# Callback to update dashboard components
@app.callback(
    [Output("room-selector", "options"),
     Output("room-selector", "value"),
     Output("summary-overview", "children"),
     Output("hourly-trends", "figure"),
     Output("weekly-trends", "figure"),
     Output("heatmap", "figure"),
     Output("anomalies", "figure")],
    [Input("floor-selector", "value"),
     Input("room-selector", "value")]
)
def update_dashboard(selected_floor, selected_rooms):
    # Select data based on the floor
    if selected_floor == "ground":
        room_data = ground_floor
        hourly_data = ground_hourly
        weekly_data = ground_weekly
        heatmap_data = ground_heatmap
        anomalies_data = ground_anomalies
        room_options = [{"label": col, "value": col} for col in room_columns_ground]
    else:
        room_data = first_floor
        hourly_data = first_hourly
        weekly_data = first_weekly
        heatmap_data = first_heatmap
        anomalies_data = first_anomalies
        room_options = [{"label": col, "value": col} for col in room_columns_first]

    # Aggregate if no specific rooms selected
    if not selected_rooms:
        hourly_avg = hourly_data.mean(axis=1).reset_index()
        weekly_avg = weekly_data.mean(axis=1).reset_index()
        heatmap_avg = heatmap_data.groupby(['Hour', 'Weekday']).mean().reset_index()
        anomalies_avg = anomalies_data.mean(axis=1).reset_index()
        room_label = "All Rooms"
    else:
        hourly_avg = hourly_data[selected_rooms].mean(axis=1).reset_index()
        weekly_avg = weekly_data[selected_rooms].mean(axis=1).reset_index()
        heatmap_avg = heatmap_data.groupby(['Hour', 'Weekday']).mean().reset_index()
        anomalies_avg = anomalies_data[selected_rooms].mean(axis=1).reset_index()
        room_label = ", ".join(selected_rooms)

    # Generate visualizations
    fig_hourly = px.line(hourly_avg, x="Hour", y=hourly_avg.columns[1],
                         title=f"Hourly Trends - {room_label}",
                         labels={"Hour": "Hour of Day", hourly_avg.columns[1]: "Utilization"},
                         template="plotly_white")
    
    fig_hourly.update_layout(
    yaxis=dict(
        title="Utilization (%)",  # Correct y-axis label
        showgrid=True,  # Optional: Keep gridlines for clarity
        zeroline=False,  # Disable the zero line
        range=[hourly_avg[hourly_avg.columns[1]].min() - 0.05, hourly_avg[hourly_avg.columns[1]].max() + 0.05]  # Dynamically adjust the range
    )
    )

    fig_weekly = px.bar(weekly_avg, x="Weekday", y=weekly_avg.columns[1],
                        title=f"Weekly Trends - {room_label}",
                        labels={"Weekday": "Day of the Week", weekly_avg.columns[1]: "Utilization"},
                        template="plotly_white")
    
    fig_weekly.update_layout(
    yaxis=dict(
        title="Utilization (%)",  # Correct y-axis label
        showgrid=True,  # Optional: Keep gridlines for clarity
        zeroline=False,  # Disable the zero line
        range=[weekly_avg[weekly_avg.columns[1]].min() - 0.05, weekly_avg[weekly_avg.columns[1]].max() + 0.05]  # Dynamically adjust the range
    )
    )

    fig_heatmap = px.density_heatmap(
        heatmap_data.groupby(['Hour', 'Weekday']).mean().reset_index(),
        x="Hour", y="Weekday", z=heatmap_data.columns[-1],
        title=f"Heatmap - All Rooms ({selected_floor.capitalize()} Floor)",
        labels={"Hour": "Hour of Day", "Weekday": "Day of the Week", heatmap_data.columns[-1]: "Utilization"},
        template="plotly_white")

    fig_anomalies = px.scatter(anomalies_avg, x=anomalies_avg.columns[0], y=anomalies_avg.columns[1],
                               title=f"Anomalies - {room_label}",
                               labels={"Index": "Time Index", "Z-Score": "Anomaly Score"},
                               template="plotly_white")
    
    fig_anomalies.update_layout(
    yaxis=dict(
        title="Anomaly Score",  # Correct y-axis label
        showgrid=True,  # Optional: Keep gridlines for clarity
        zeroline=False,  # Disable the zero line
        range=[
            anomalies_avg[anomalies_avg.columns[1]].min() - 0.05, 
            anomalies_avg[anomalies_avg.columns[1]].max() + 0.05
        ]  # Dynamically adjust the range
    ),
    xaxis=dict(
        title="Time Index",  # Correct x-axis label
        showgrid=True  # Optional: Keep gridlines for clarity
    )
    )

    # Summary
    summary = f"Overview: Average Utilization {hourly_avg[hourly_avg.columns[1]].mean():.2f}%"

    return room_options, None, summary, fig_hourly, fig_weekly, fig_heatmap, fig_anomalies

# Run the app
if __name__ == "__main__":
    print("Visit http://127.0.0.1:8050/ to access the dashboard.")
    app.run_server(debug=True)